In [5]:
import pandas as pd
import numpy as np
import datetime
import os

In [6]:
fs=[f for f in os.listdir('data') if '_data.csv' in f]

In [7]:
def get_df(fn,start,end):
    df = pd.read_csv('data/{}'.format(fn))
    df.index = pd.to_datetime(df['time'])
    df=df.sort_index()
    g = df.groupby(df.index.date)
    to_return = []
    for k,v in g.groups.items():
        if k.month==7 and k.day>=10:
            #print(k)
            day_dict = df.loc[v]
            day_dict = day_dict.loc[(day_dict.index.hour>=start) & (day_dict.index.hour<=end)]
            to_return.append(day_dict)
    
    return to_return

In [97]:
def get_percent_available_at_beginning_of_run(d):
    xt = pd.Series(d.loc[(d['online_state']==1)]['available'])
    runs = [[10,14],[15,19],[20,23]]
    alls = []
    for r in runs:
        temp = [val for row,val in xt.iteritems() if row.hour>=r[0] and row.hour<r[1]]
        if len(temp)>0:
            alls.append(np.array(temp).mean())
    if len(alls)>0:
        return np.array(alls).mean()
    return -100
         

In [98]:
def get_number_available_at_beginning_of_run(d):
    xt = pd.Series(d.loc[ (d['online_state']==1)]['available'])
    runs = [[10,14],[15,19],[20,23]]
    alls = []
    for r in runs:
        temp = [val for row,val in xt.iteritems() if row.hour>=r[0] and row.hour<r[1]]
        if len(temp)>0:
            alls.append(sum(temp))
    if len(alls)>0:
        return np.array(alls).mean()
    return -100
         

In [99]:
def get_percent_sent_at_beginning_of_run(d):
    xt = pd.Series(d.loc[(d['available']==1 )& (d['online_state']==1)]['action'])
    runs = [[10,14],[15,19],[20,23]]
    alls = []
    for r in runs:
        temp = [val for row,val in xt.iteritems() if row.hour>=r[0] and row.hour<r[1]]
        if len(temp)>0:
            alls.append(np.array(temp).mean())
    if len(alls)>0:
        return np.array(alls).mean()
    return -100

In [100]:
def get_percent_available_at_end_of_run(d):
    xt = pd.Series(d.loc[ (d['online_state']==1)]['available'])
    runs = [[14,15],[19,20],[23,24]]
    alls = []
    for r in runs:
        temp = [val for row,val in xt.iteritems() if row.hour>=r[0] and row.hour<r[1]]
        if len(temp)>0:
            alls.append(np.array(temp).mean())
    if len(alls)>0:
        return np.array(alls).mean()
    return -100
         

In [101]:
def get_number_available_at_end_of_run(d):
    xt = pd.Series(d.loc[ (d['online_state']==1)]['available'])
    runs = [[14,15],[19,20],[23,24]]
    alls = []
    for r in runs:
        temp = [val for row,val in xt.iteritems() if row.hour>=r[0] and row.hour<r[1]]
        if len(temp)>0:
            alls.append(sum(temp))
    if len(alls)>0:
        return np.array(alls).mean()
    return -100

In [89]:
def get_percent_sent_at_end_of_run(d):
    xt = pd.Series(d.loc[(d['available']==1 )& (d['online_state']==1)]['action'])
    runs = [[10,15],[15,20],[20,24]]
    alls = []
    for r in runs:
        temp = [val for row,val in xt.iteritems() if row.hour>=r[0] and row.hour<r[1]]
        if len(temp)>0:
            alls.append(np.array(temp).mean())
    if len(alls)>0:
        return np.array(alls).mean()
    return -100
         

In [90]:
def get_all_user(fn,start,end):
    df = get_df(fn,start,end)
    fractions = [get_percent_available_at_end_of_run(d) for d in df if len(d.index)>0]
    fractions = np.array([f for f in fractions if f!=-100])
    return fractions.mean()

In [91]:
def compare_beginning_end(fn,start,end):
    df = get_df(fn,start,end)
    fractions = [get_percent_available_at_end_of_run(d) for d in df if len(d.index)>0]
    fractions = np.array([f for f in fractions if f!=-100])
    end = fractions.mean()
    
    fractions = [get_percent_available_at_beginning_of_run(d) for d in df if len(d.index)>0]
    fractions = np.array([f for f in fractions if f!=-100])
    start = fractions.mean()
    
    return [start,end]

In [92]:
def compare_beginning_end_actions(fn,start,end):
    df = get_df(fn,start,end)
    fractions = [get_percent_sent_at_end_of_run(d) for d in df if len(d.index)>0]
    fractions = np.array([f for f in fractions if f!=-100])
    end = fractions.mean()
    
    fractions = [get_percent_sent_at_beginning_of_run(d) for d in df if len(d.index)>0]
    fractions = np.array([f for f in fractions if f!=-100])
    start = fractions.mean()
    
    return [start,end]

In [93]:
def get_runs(series):
    alls = []
    temp = []
    for i in range(len(series)):
        if series[i]==1:
            temp.append(series[i])
        else:
            if len(temp)>0:
                alls.append(temp)
                temp=[]
    if len(temp)>0:
        alls.append(temp)
    return alls

In [94]:
def runs(fn,start,end):
    df = get_df(fn,start,end)
    runs = []
    for d in df:
        if len(d.index)>0:
            runs.extend(get_runs(d['online_state']))
    lens = [len(r) for r in runs]
    if sum(lens)==0:
        return 0
    return np.array(lens).mean()

In [95]:
def compare_beginning_end_number(fn,start,end):
    df = get_df(fn,start,end)
    fractions = [get_number_available_at_end_of_run(d) for d in df if len(d.index)>0]
    fractions = np.array([f for f in fractions if f!=-100])
    end = fractions.mean()
    
    fractions = [get_number_available_at_beginning_of_run(d) for d in df if len(d.index)>0]
    fractions = np.array([f for f in fractions if f!=-100])
    start = fractions.mean()
    
    return [start,end]

In [102]:
average_sent  = [[],[]]
average_percent = [[],[]]
average_number = [[],[]]
for fn in fs:
    #print(fn)
    cone = compare_beginning_end(fn,9,21)
    ctwo = compare_beginning_end_actions(fn,9,21)
    cthree = compare_beginning_end_number(fn,9,21)
    if 'pedja' in fn:
        print(fn)
        print(cone)
        print(ctwo)
        print(cthree)
        print('\n')
    if 'pedja' not in fn and 'mash' not in fn and 'walter' not in fn and 'user_2' not in fn:
        average_percent[0].append(cone[0])
        average_sent[0].append(ctwo[0])
        average_number[0].append(cthree[0])
        average_percent[1].append(cone[1])
        average_sent[1].append(ctwo[1])
        average_number[1].append(cthree[1])
print([np.array(a).mean() for a in average_percent ])
print([np.array(a).mean() for a in average_sent ])
print([np.array(a).mean() for a in average_number ])

user_test-pedja_antised_data.csv
[0.5631112351582005, 0.7663930976430976]
[0.15869577952911285, 0.1728771889486175]
[6.380952380952381, 4.416666666666667]


[0.6633789865702228, 0.5980735491998024]
[0.1453591151573373, 0.13404726116768145]
[8.690106619570907, 3.411529051566645]


In [2]:
df = pd.read_csv('data/user_test-pedja_antised_data.csv')

In [83]:
aruns = []
#average_percent = [[],[]]
#average_number = [[],[]]
for fn in fs:
    #print(fn)
    cone = runs(fn,9,21)

    if 'pedja' in fn:
        print(fn)
        print(cone)
        print('\n')
    if 'pedja' not in fn and 'mash' not in fn and 'walter' not in fn and 'user_2' not in fn:
        aruns.append(cone)
#print(aruns)
print(np.array(aruns).mean() )


user_test-pedja_antised_data.csv
7.28125


9.337154352531526


In [4]:
g = df.groupby(df.index.date)

In [5]:
for k,v in g.groups.items():
    print(k)
    print(df.loc[v])

2019-04-30
                         userid decisionid                 time  \
time                                                              
2019-04-30 09:00:00  test-pedja       8800  2019-04-30 09:00:00   
2019-04-30 15:00:00  test-pedja       6866  2019-04-30 15:00:00   

                                daystart               dayend  online_state  \
time                                                                          
2019-04-30 09:00:00  2019-04-30 02:00:00  2019-04-30 14:00:00             0   
2019-04-30 15:00:00  2019-04-30 08:00:00  2019-04-30 20:00:00             0   

                     online_step  available  batch_state  batch_step  \
time                                                                   
2019-04-30 09:00:00            8          0           -1          -1   
2019-04-30 15:00:00            8          0           -1          -1   

                     probaction  action  missingindicator  duplicate  
time                                       

                         userid decisionid                 time  \
time                                                              
2019-05-18 08:05:00  test-pedja       7418  2019-05-18 08:05:00   
2019-05-18 08:10:00  test-pedja       1414  2019-05-18 08:10:00   
2019-05-18 08:15:00  test-pedja       1337  2019-05-18 08:15:00   
2019-05-18 08:20:00  test-pedja       6399  2019-05-18 08:20:00   
2019-05-18 08:25:00  test-pedja       2802  2019-05-18 08:25:00   
2019-05-18 08:30:00  test-pedja       6307  2019-05-18 08:30:00   
2019-05-18 08:35:00  test-pedja       7454  2019-05-18 08:35:00   
2019-05-18 08:40:00  test-pedja       3350  2019-05-18 08:40:00   
2019-05-18 08:45:00  test-pedja       4487  2019-05-18 08:45:00   
2019-05-18 08:50:00  test-pedja       6124  2019-05-18 08:50:00   
2019-05-18 08:55:00  test-pedja        306  2019-05-18 08:55:00   
2019-05-18 09:00:00  test-pedja       7559  2019-05-18 09:00:00   
2019-05-18 09:05:00  test-pedja       3861  2019-05-18 09:05:0

                         userid decisionid                 time  \
time                                                              
2019-06-04 02:00:00  test-pedja        971  2019-06-04 02:00:00   
2019-06-04 02:05:00  test-pedja        883  2019-06-04 02:05:00   
2019-06-04 02:10:00  test-pedja       8803  2019-06-04 02:10:00   
2019-06-04 02:15:00  test-pedja       3887  2019-06-04 02:15:00   
2019-06-04 02:20:00  test-pedja       8475  2019-06-04 02:20:00   
2019-06-04 02:25:00  test-pedja       5138  2019-06-04 02:25:00   
2019-06-04 02:30:00  test-pedja       5684  2019-06-04 02:30:00   
2019-06-04 02:35:00  test-pedja       6332  2019-06-04 02:35:00   
2019-06-04 02:40:00  test-pedja        384  2019-06-04 02:40:00   
2019-06-04 02:45:00  test-pedja       8173  2019-06-04 02:45:00   
2019-06-04 02:50:00  test-pedja       9517  2019-06-04 02:50:00   
2019-06-04 02:55:00  test-pedja       3255  2019-06-04 02:55:00   
2019-06-04 03:00:00  test-pedja        141  2019-06-04 03:00:0

                         userid decisionid                 time  \
time                                                              
2019-06-17 09:05:00  test-pedja       3940  2019-06-17 09:05:00   
2019-06-17 09:10:00  test-pedja       1470  2019-06-17 09:10:00   
2019-06-17 09:15:00  test-pedja       7945  2019-06-17 09:15:00   
2019-06-17 09:20:00  test-pedja       8269  2019-06-17 09:20:00   
2019-06-17 09:25:00  test-pedja       9045  2019-06-17 09:25:00   
2019-06-17 09:30:00  test-pedja       5086  2019-06-17 09:30:00   
2019-06-17 09:35:00  test-pedja       6493  2019-06-17 09:35:00   
2019-06-17 09:40:00  test-pedja       2577  2019-06-17 09:40:00   
2019-06-17 09:45:00  test-pedja       1168  2019-06-17 09:45:00   
2019-06-17 09:50:00  test-pedja       2437  2019-06-17 09:50:00   
2019-06-17 09:55:00  test-pedja       8361  2019-06-17 09:55:00   
2019-06-17 10:00:00  test-pedja       1109  2019-06-17 10:00:00   
2019-06-17 10:05:00  test-pedja       4076  2019-06-17 10:05:0

                         userid decisionid                 time  \
time                                                              
2019-06-28 05:05:00  test-pedja        868  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja        610  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja       8777  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja       7913  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja       3735  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja       4025  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja       5995  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja       3343  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja        868  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja        610  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja       8777  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja       7913  2019-06-28 05:05:00   
2019-06-28 05:05:00  test-pedja       3735  2019-06-28 05:05:0

                         userid decisionid                 time  \
time                                                              
2019-07-12 08:35:00  test-pedja        370  2019-07-12 08:35:00   
2019-07-12 09:05:00  test-pedja       1698  2019-07-12 09:05:00   
2019-07-12 09:10:00  test-pedja       5935  2019-07-12 09:10:00   
2019-07-12 09:15:00  test-pedja       6085  2019-07-12 09:15:00   
2019-07-12 09:20:00  test-pedja       2723  2019-07-12 09:20:00   
2019-07-12 09:25:00  test-pedja       5201  2019-07-12 09:25:00   
2019-07-12 09:30:00  test-pedja       6062  2019-07-12 09:30:00   
2019-07-12 09:35:00  test-pedja       7806  2019-07-12 09:35:00   
2019-07-12 09:40:00  test-pedja       8708  2019-07-12 09:40:00   
2019-07-12 09:45:00  test-pedja       5622  2019-07-12 09:45:00   
2019-07-12 09:50:00  test-pedja       5041  2019-07-12 09:50:00   
2019-07-12 09:55:00  test-pedja       5963  2019-07-12 09:55:00   
2019-07-12 10:00:00  test-pedja       3136  2019-07-12 10:00:0

                         userid decisionid                 time  \
time                                                              
2019-07-26 09:00:00  test-pedja       3084  2019-07-26 09:00:00   
2019-07-26 09:05:00  test-pedja       3659  2019-07-26 09:05:00   
2019-07-26 09:10:00  test-pedja       9864  2019-07-26 09:10:00   
2019-07-26 09:15:00  test-pedja       9911  2019-07-26 09:15:00   
2019-07-26 09:20:00  test-pedja       4384  2019-07-26 09:20:00   
2019-07-26 09:25:00  test-pedja       9755  2019-07-26 09:25:00   
2019-07-26 09:30:00  test-pedja       5879  2019-07-26 09:30:00   
2019-07-26 09:35:00  test-pedja       5405  2019-07-26 09:35:00   
2019-07-26 09:40:00  test-pedja       9805  2019-07-26 09:40:00   
2019-07-26 09:45:00  test-pedja       9697  2019-07-26 09:45:00   
2019-07-26 09:50:00  test-pedja       9265  2019-07-26 09:50:00   
2019-07-26 09:55:00  test-pedja        226  2019-07-26 09:55:00   
2019-07-26 10:00:00  test-pedja       9729  2019-07-26 10:00:0

In [55]:
pd.Timestamp(datetime.date(2019, 4, 30))

Timestamp('2019-04-30 00:00:00')

In [71]:
five_day = df[g.groups[datetime.date(2019, 7, 24)][0]:g.groups[datetime.date(2019, 7, 24)][-1]]['available']


NameError: name 'df' is not defined

In [18]:
five_act = df[g.groups[datetime.date(2019, 7, 24)][0]:g.groups[datetime.date(2019, 7, 24)][-1]]['action']


In [70]:
get_runs(five_day['2019-07-24 15:00:00':'2019-07-24 17:55:00'])

NameError: name 'five_day' is not defined

In [20]:
five_day['2019-07-24 15:00:00':'2019-07-24 17:55:00']

time
2019-07-24 15:00:00    0
2019-07-24 15:05:00    0
2019-07-24 15:10:00    0
2019-07-24 15:15:00    0
2019-07-24 15:20:00    0
2019-07-24 15:25:00    0
2019-07-24 15:30:00    1
2019-07-24 15:35:00    1
2019-07-24 15:40:00    1
2019-07-24 15:45:00    1
2019-07-24 15:50:00    1
2019-07-24 15:55:00    1
2019-07-24 16:00:00    0
2019-07-24 16:05:00    0
2019-07-24 16:10:00    0
2019-07-24 16:15:00    0
2019-07-24 16:20:00    0
2019-07-24 16:25:00    0
2019-07-24 16:30:00    0
2019-07-24 16:35:00    0
2019-07-24 16:40:00    0
2019-07-24 16:45:00    0
2019-07-24 16:50:00    0
2019-07-24 16:55:00    1
2019-07-24 17:00:00    1
2019-07-24 17:10:00    0
2019-07-24 17:15:00    0
2019-07-24 17:20:00    0
2019-07-24 17:25:00    0
2019-07-24 17:30:00    0
2019-07-24 17:35:00    0
2019-07-24 17:40:00    0
2019-07-24 17:45:00    0
2019-07-24 17:50:00    0
2019-07-24 17:55:00    0
Name: available, dtype: int64

In [83]:
for day in range(18,25):
    print(day)
    seds = df[g.groups[datetime.date(2019, 7, day)][0]:g.groups[datetime.date(2019, 7, day)][-1]]['online_state']
    print(sum(seds)/len(seds))




18
0.39849624060150374
19
0.2366412213740458
20
0.21014492753623187
21
0.078125
22
0.37815126050420167
23
0.4461538461538462
24
0.38686131386861317


In [82]:
sum(seds)/len(seds)

0.37815126050420167